# Direct Database Access for Devs
author: Grant Saggars,
credits: Grant Saggars,
maintainer: Grant Saggars,
email: g293s490@ku.edu

Note: querying is designed with the web in mind, and these queries can be done for real with the web API (recommended, see `app/routers/vX_api.py`). This notebook seeks to demonstrate how basic queries are done with the minimal python involved, and output may not be very human-readable, especially compared to the web output.

In [2]:
# As usual, do imports
from typing import Any, List, Optional
from dotenv import load_dotenv
import psycopg
from psycopg.rows import class_row
from pydantic import BaseModel, Field
from pydantic_settings import BaseSettings

Now, we may define a class (inheriting from pydantic BaseModel to be used as a row factory) having every parameter we want to grab from the query. Simply commenting a line out will cause it to not appear when queried.

In [3]:
class Material(BaseModel):
    """Class exposing the parameters for queries on our material."""
    id: Optional[int] = None
    search_vector: Any
    MP_ID: Optional[str] = Field(alias="MP-ID")
    formula: Optional[str]
    spacegroup: Optional[str]
    cell: Optional[dict]  # JSONB
    symbols: Optional[List[str]]
    positions: Optional[List[List[float]]]
    vdw_gap: Optional[float] = Field(alias="vdw gap")
    bond_length_deviation: Optional[Any] = Field(alias="bond length deviation")  # JSONB
    bond_angle_deviation: Optional[Any] = Field(alias="bond angle deviation")    # JSONB
    mass_density: Optional[float] = Field(alias="mass density")
    total_energy: Optional[float] = Field(alias="total energy")
    total_energy_soc: Optional[float] = Field(alias="total energy_soc")
    cohesive_energy: Optional[float] = Field(alias="cohesive energy")
    exfoliation_energy: Optional[float] = Field(alias="exfoliation energy")
    born_effective_charge_tensor: Optional[float] = Field(alias="born effective charge tensor")
    born_effective_charge_q_xy: Optional[float] = Field(alias="born effective charge q_xy")
    born_effective_charge_q_z: Optional[float] = Field(alias="born effective charge q_z")
    dielectric_constant_XY: Optional[float] = Field(alias="dielectric constant XY")
    dielectric_constant_Z: Optional[float] = Field(alias="dielectric constant Z")
    bader_charge: Optional[Any] = Field(alias="bader charge")  # JSONB
    density_of_states_at_fermi: Optional[float] = Field(alias="density of states at fermi")
    effective_mass: Optional[float] = Field(alias="effective mass")
    vbm: Optional[float]
    cbm: Optional[float]
    band_gap: Optional[float] = Field(alias="band gap")
    vbm_soc: Optional[float] = Field(alias="vbm soc")
    cbm_soc: Optional[float] = Field(alias="cbm soc")
    band_gap_soc: Optional[float] = Field(alias="band gap soc")
    layered: Optional[bool] = Field(alias="layered?")
    component_layers: Optional[List[str]] = Field(alias="component layers")
    KPath: Optional[List[str]]
    band_locations: Optional[str] = Field(alias="band locations")
    band_soc_location: Optional[str] = Field(alias="band soc location")
    dos_location: Optional[str] = Field(alias="dos location")

We query the database with simple PSQL. In the future, more complex queries may be implemented (namely searching with filtering). Please refer to the [postgres documentation](https://www.postgresql.org/docs/current/app-psql.html) for more information on the syntax and available commands!

In [9]:
def get_all(conninfo: str) -> list[Material]:
    with psycopg.connect(conninfo) as conn, conn.cursor(
        row_factory=class_row(Material)
    ) as cur:
        cur.execute("select * from material_data")
        records = cur.fetchall()
        return records

def search_contains(conninfo: str, query: str, limit: int = 20) -> Any:
    with psycopg.connect(conninfo) as conn, conn.cursor(
        row_factory=class_row(Material)
    ) as cur:
        cur.execute("""
            SELECT *,
                   CASE 
                       WHEN formula = %s THEN 100
                       WHEN formula ILIKE %s THEN 100
                       WHEN "MP-ID" = %s THEN 90
                       WHEN "MP-ID" ILIKE %s THEN 70
                       WHEN "spacegroup" = %s THEN 100
                       WHEN "spacegroup" ILIKE %s THEN 70
                       ELSE 50
                   END as relevance_score
            FROM material_data 
            WHERE formula ILIKE %s 
               OR "MP-ID" ILIKE %s
               OR "spacegroup" ILIKE %s
            ORDER BY relevance_score DESC, formula
            LIMIT %s
        """, [
                  query, query, query, query, query, query,  # relevance scoring
                  f'%{query}%', f'%{query}%', f'%{query}%',  # where conditions
                  limit
              ])

        records = cur.fetchall()
        return records

def get(conninfo: str, id: int) -> Material | None:
    with psycopg.connect(conninfo) as conn, conn.cursor(
        row_factory=class_row(Material)
    ) as cur:
        cur.execute("select * from material_data where id=%s", [id])
        record = cur.fetchone()
        return record

def delete(conninfo: str, id: int):
    with psycopg.connect(conninfo) as conn:
        conn.execute("delete from material_data where id=%s", [id])

For the sake of security, I'll grab our psql connection settings from the .env file:

In [15]:
class Settings(BaseSettings):
    db_host: str
    db_user: str
    db_password: str
    db_port: str
    db_name: str

    class Config:
        env_file = ".env"

load_dotenv() # force load the .env file

# settings defined in our .env for this; obfuscated for security.
settings: Settings = Settings()  # type: ignore
conninfo = f"user={settings.db_user} password={settings.db_password} host={settings.db_host} port={settings.db_port} dbname={settings.db_name}"

Now, to showcase some of these things:

In [16]:
print(search_contains(conninfo, "Bi"))

[Material(id=127, search_vector="'-27916':3A '166':7B '3m':6B 'ag':8C,9C,10C 'ag3bi3se6':1A 'bi':11C,12C,13C 'mp':2A 'r':5B 'r-3m':4B 'se':14C,15C,16C,17C,18C,19C", MP_ID='mp-27916', formula='Ag3Bi3Se6', spacegroup='R-3m (166)', cell={'array': [[4.159614760218917, 0.0, 0.0], [-2.0798073801094574, 3.602332052306299, 0.0], [0.0, 0.0, 19.583062151747633]]}, symbols=['Ag', 'Ag', 'Ag', 'Bi', 'Bi', 'Bi', 'Se', 'Se', 'Se', 'Se', 'Se', 'Se'], positions=[[0.0, 0.0, 0.0], [2.0798074009075194, 1.2007773387610003, 6.527687318639045], [-2.0798060060594915e-08, 2.401554713545299, 13.055374833108589], [-2.0798060060594915e-08, 2.401554713545299, 3.263843757234772], [0.0, 0.0, 9.791531075873817], [2.0798074009075194, 1.2007773387610003, 16.319218394512863], [0.0, 0.0, 4.989811110408869], [2.0798074009075194, 1.2007773387610003, 1.5378764040606745], [2.0798074009075194, 1.2007773387610003, 11.51749842904791], [-2.0798060060594915e-08, 2.401554713545299, 8.06556372269972], [-2.0798060060594915e-08, 2.40

In [17]:
print(get(conninfo, 32))  # Get the 32nd item in our database

id=32 search_vector="'164':5B '3m1':4B 'p':3B 'p-3m1':2B 'sb':6C,7C 'sb2se2te':1A 'se':9C,10C 'te':8C" MP_ID=None formula='Sb2Se2Te' spacegroup='P-3m1 (164)' cell={'array': [[4.101695191631345, -4e-16, 0.0], [-6.152542787447018, -3.5521722345332245, 0.0], [0.0, 0.0, -27.16439394068898]]} symbols=['Sb', 'Sb', 'Te', 'Se', 'Se'] positions=[[0.0, 0.0, -11.505690847934225], [-2.0508475958156875, -1.1840574351922335, -15.65870309275475], [-4.101695191631331, -2.3681148703844417, -13.582196816756769], [-2.0508475958156875, -1.1840574351922335, -10.007921271347918], [0.0, 0.0, -17.156472669341063]] vdw_gap=2.076506275997982 bond_length_deviation=[['Sb', 1.1304785837105138e-07], ['Se', 1.550283272921666e-08]] bond_angle_deviation=[['Sb', 3.4792396954730914e-06], ['Se', 3.40509955541072e-07]] mass_density=3.935914016139414 total_energy=-126.36823759 total_energy_soc=None cohesive_energy=-4.183052271625002 exfoliation_energy=None born_effective_charge_tensor=13.34182 born_effective_charge_q_xy=6.

In [18]:
print(get_all(conninfo))

[Material(id=1, search_vector="'164':5B '3m1':4B 'p':3B 'p-3m1':2B 'sb':6C,7C 'sb2':1A", MP_ID=None, formula='Sb2', spacegroup='P-3m1 (164)', cell={'array': [[4.089723574509989, 1e-16, 0.0], [2.044861787254994, 3.5418045099817506, 0.0], [0.0, 0.0, 21.464900137019256]]}, symbols=['Sb', 'Sb'], positions=[[0.0, 0.0, 11.545738142954642], [2.0448617668063895, 1.1806014915212426, 9.919161994064613]], vdw_gap=1.6265761488900292, bond_length_deviation=[['Sb', 0.5929946479518043]], bond_angle_deviation=[['Sb', 26.2696831731489]], mass_density=5.167446971323506, total_energy=-49.75763561, total_energy_soc=None, cohesive_energy=0.8274979249999994, exfoliation_energy=None, born_effective_charge_tensor=0.0, born_effective_charge_q_xy=0.0, born_effective_charge_q_z=0.0, dielectric_constant_XY=6.336829, dielectric_constant_Z=1.267907, bader_charge=[['Sb', 5.0, 5.002576], ['Sb', 5.0, 4.997424]], density_of_states_at_fermi=0.006347, effective_mass=None, vbm=1.2751, cbm=-2.6923, band_gap=-1.4172, vbm_so